In [121]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [30]:
df = pd.read_json('../data/wikidata-movies.json.gz', orient='records', lines=True)
genres = pd.read_json('../data/genres.json.gz', orient='records', lines=True)

In [31]:
df = df[pd.notnull(df['made_profit'])]

In [24]:
df['genre'] = df['genre'].str[0]
genres.columns = ['genre_label', 'genre']

In [25]:
merged = pd.merge(df, genres, on="genre")

In [55]:
df_stack = df['genre'].apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame()
df_stack.columns = ['wikidata_id']

In [65]:
df_genre = df_stack.reset_index().merge(genres, how="left", on="wikidata_id").set_index('index')

In [83]:
df_genre = pd.merge(df_genre, df, left_index=True, right_index=True).reset_index()

In [114]:
genres['profit'] = genres['genre_label'].apply(lambda x: df_genre[(df_genre['genre_label'] == x) & (df_genre['made_profit'] == True)].shape[0])
genres['noprofit'] = genres['genre_label'].apply(lambda x: df_genre[(df_genre['genre_label'] == x) & (df_genre['made_profit'] == False)].shape[0])

In [108]:
df_genre[(df_genre['country_of_origin'] == 'Q29') & (df_genre['made_profit'] == True)].shape[0]

7

In [117]:
df_genre = genres.sort_values(by='profit', ascending=False)
df_genre = df_genre[(df_genre['profit'] != 0) & (df_genre['noprofit'] != 0)]

In [119]:
# profit_percent = profit/total
df_genre['profit_percent'] = df_genre['profit'] / (df_genre['profit'] + df_genre['noprofit'])

In [1]:
df_genre['profit']

In [146]:
contingency = np.vstack([df_genre['profit'], df_genre['noprofit']])

In [147]:
chi2, p, dof, expected = stats.chi2_contingency(contingency)

In [148]:
p

0.008127575360670712